# Compare NLP Techniques: Build Model On word2vec Vectors

### Read In Cleaned Text

In [11]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('../../../data/X_train.csv')
X_test = pd.read_csv('../../../data/X_test.csv')
y_train = pd.read_csv('../../../data/y_train.csv')
y_test = pd.read_csv('../../../data/y_test.csv')

### Create word2vec Vectors

In [15]:
# Train a basic word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [21]:
# Replace the words in each text message with the learned word vector
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                        for ls in X_train['clean_text']], dtype=object)
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test['clean_text']], dtype=object)

In [23]:
# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [26]:
# What does the unaveraged version look like?
X_train_vect[0]

array([[-8.6196875e-03,  3.6657380e-03,  5.1898835e-03,  5.7419385e-03,
         7.4669183e-03, -6.1676754e-03,  1.1056137e-03,  6.0472824e-03,
        -2.8400505e-03, -6.1735227e-03, -4.1022300e-04, -8.3689485e-03,
        -5.6000124e-03,  7.1045388e-03,  3.3525396e-03,  7.2256695e-03,
         6.8002474e-03,  7.5307419e-03, -3.7891543e-03, -5.6180597e-04,
         2.3483764e-03, -4.5190323e-03,  8.3887316e-03, -9.8581640e-03,
         6.7646410e-03,  2.9144168e-03, -4.9328315e-03,  4.3981876e-03,
        -1.7395747e-03,  6.7113843e-03,  9.9648498e-03, -4.3624435e-03,
        -5.9933780e-04, -5.6956373e-03,  3.8508223e-03,  2.7866268e-03,
         6.8910765e-03,  6.1010956e-03,  9.5384968e-03,  9.2734173e-03,
         7.8980681e-03, -6.9895042e-03, -9.1558648e-03, -3.5575271e-04,
        -3.0998408e-03,  7.8943167e-03,  5.9385742e-03, -1.5456629e-03,
         1.5109634e-03,  1.7900408e-03,  7.8175711e-03, -9.5101865e-03,
        -2.0553112e-04,  3.4691966e-03, -9.3897223e-04,  8.38177

In [27]:
# What does the averaged version look like?
X_train_vect_avg[0]

array([-8.6196875e-03,  3.6657383e-03,  5.1898835e-03,  5.7419385e-03,
        7.4669183e-03, -6.1676754e-03,  1.1056137e-03,  6.0472824e-03,
       -2.8400505e-03, -6.1735227e-03, -4.1022300e-04, -8.3689485e-03,
       -5.6000124e-03,  7.1045393e-03,  3.3525396e-03,  7.2256690e-03,
        6.8002478e-03,  7.5307423e-03, -3.7891543e-03, -5.6180597e-04,
        2.3483764e-03, -4.5190323e-03,  8.3887316e-03, -9.8581640e-03,
        6.7646415e-03,  2.9144168e-03, -4.9328315e-03,  4.3981876e-03,
       -1.7395748e-03,  6.7113847e-03,  9.9648498e-03, -4.3624435e-03,
       -5.9933780e-04, -5.6956373e-03,  3.8508221e-03,  2.7866268e-03,
        6.8910765e-03,  6.1010956e-03,  9.5384968e-03,  9.2734173e-03,
        7.8980681e-03, -6.9895042e-03, -9.1558648e-03, -3.5575271e-04,
       -3.0998408e-03,  7.8943167e-03,  5.9385742e-03, -1.5456629e-03,
        1.5109634e-03,  1.7900408e-03,  7.8175711e-03, -9.5101865e-03,
       -2.0553111e-04,  3.4691966e-03, -9.3897228e-04,  8.3817719e-03,
      

### Fit RandomForestClassifier On Top Of Word Vectors

In [28]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [29]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)

In [30]:
# Evaluate the predictions of the model on the holdout test set
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.649 / Recall: 0.216 / Accuracy: 0.862
